In [1]:
import pandas as pd

In [2]:
df_stock_exchange = pd.read_excel('../data/raw/borsa_ist.xlsx')
df_bitcoin = pd.read_excel('../data/raw/btc.xlsx')
df_gold = pd.read_excel('../data/raw/gold_prices.xlsx')
df_gov_bonds = pd.read_excel('../data/raw/gov_bonds.xlsx')
df_hpi = pd.read_excel('../data/raw/hpi.xlsx')
df_cpi = pd.read_csv('../data/raw/turkish_inflation_cpi.csv', sep=";")

# Stock_Exchange:

In [3]:
df_stock_exchange = df_stock_exchange.iloc[1:].reset_index(drop=True)
df_stock_exchange.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df_stock_exchange.rename(columns={'Last Price (TRY)': 'Price_Stock_Ex'}, inplace=True)
df_stock_exchange['Date'] = pd.to_datetime(df_stock_exchange['Date'], errors='coerce')
df_stock_exchange['Date'] = df_stock_exchange['Date'].dt.strftime('%Y-%m')

df_stock_exchange['stock_return_mo'] = df_stock_exchange['Price_Stock_Ex'] / df_stock_exchange['Price_Stock_Ex'].shift(-1) - 1 #calculate 1 month stock return
df_stock_exchange['stock_return_pa'] = (1 + df_stock_exchange['stock_return_mo'])**(12) - 1 #calculate stock return per annum

df_stock_exchange
df_stock_exchange.to_csv('../data/processed/df_stock_exchange.csv', index=False)

# Bitcoin:

In [4]:
df_bitcoin = df_bitcoin.iloc[1:].reset_index(drop=True)
df_bitcoin.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df_bitcoin.rename(columns={'Unnamed: 1': 'Price_Bitcoin'}, inplace=True)
df_bitcoin['Date'] = pd.to_datetime(df_bitcoin['Date'], errors='coerce')
df_bitcoin['Date'] = df_bitcoin['Date'].dt.strftime('%Y-%m')

df_bitcoin['btc_return_mo'] = df_bitcoin['Price_Bitcoin'] / df_bitcoin['Price_Bitcoin'].shift(-1) -1 # calculate 1 month return
df_bitcoin['btc_return_pa'] = (1 + df_bitcoin['btc_return_mo'])**(12) - 1 #calculate return per annum

df_bitcoin
df_bitcoin.to_csv('../data/processed/df_bitcoin.csv', index=False)

# Gold:

In [5]:
df_gold = df_gold.iloc[1:].reset_index(drop=True)
df_gold.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df_gold.rename(columns={'Last Price (TRY)': 'Price_Gold'}, inplace=True)
df_gold['Date'] = pd.to_datetime(df_gold['Date'], errors='coerce')
df_gold['Date'] = df_gold['Date'].dt.strftime('%Y-%m')

df_gold['gold_return_mo'] = df_gold['Price_Gold'] / df_gold['Price_Gold'].shift(-1) -1 # calculate 1 month return
df_gold['gold_return_pa'] = (1 + df_gold['gold_return_mo'])**(12) - 1 #calculate return per annum (annualized return)

df_gold
df_gold.to_csv('../data/processed/df_gold.csv', index=False)

# Government Bonds:

In [6]:
df_gov_bonds = df_gov_bonds.iloc[1:].reset_index(drop=True)
df_gov_bonds.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df_gov_bonds.rename(columns={'Yield (%)': 'Gov_Bond_Yield'}, inplace=True)
df_gov_bonds['Date'] = pd.to_datetime(df_gov_bonds['Date'], errors='coerce')
df_gov_bonds['Date'] = df_gov_bonds['Date'].dt.strftime('%Y-%m')  

df_gov_bonds['bond_return_pa'] = df_gov_bonds['Gov_Bond_Yield'] / 100

df_gov_bonds.to_csv('../data/processed/df_gov_bonds.csv', index=False)

# House Price Index:

In [7]:
df_hpi = df_hpi.iloc[1:].reset_index(drop=True)

df_hpi.rename(columns={'Tarih': 'Date'}, inplace=True)
df_hpi['Date'] = pd.to_datetime(df_hpi['Date'], errors='coerce')
df_hpi['Date'] = df_hpi['Date'].dt.strftime('%Y-%m')

df_hpi = df_hpi.dropna(subset=['Date'])

df_hpi['hpi_return_mo'] = df_hpi['TP KFE TR-1'] / 100
df_hpi['hpi_return_pa'] = (1 + df_hpi['hpi_return_mo'])**(12) - 1 #calculate annualized return

#df_hpi
df_hpi.to_csv('../data/processed/df_hpi.csv', index=False)

# Consumer Price Index

In [8]:
df_cpi = df_cpi.iloc[1:].reset_index(drop=True)

df_cpi.rename(columns={'in %': 'Inflation'}, inplace=True)
df_cpi['Date'] = pd.to_datetime(df_cpi['Date'], errors='coerce', dayfirst=True)
df_cpi['Date'] = df_cpi['Date'].dt.strftime('%Y-%m') #inflation measured at the end of the month

df_cpi['Inflation'] = df_cpi['Inflation'] / 100 

df_cpi.to_csv('../data/processed/df_cpi.csv', index=False)

# Database:

In [9]:
import sqlite3

In [10]:
conn = sqlite3.connect('../data/processed/database_name.db')

In [11]:
df_stock_exchange.to_sql('STOCKEX', conn, if_exists='replace', index=False)
df_bitcoin.to_sql('BTC', conn, if_exists='replace', index=False)
df_gold.to_sql('GOLD', conn, if_exists='replace', index=False)
df_gov_bonds.to_sql('BOND', conn, if_exists='replace', index=False)
df_hpi.to_sql("HPI",conn, if_exists='replace', index=False)
df_cpi.to_sql("CPI",conn, if_exists='replace', index=False)

80

In [12]:
query = "SELECT * FROM STOCKEX"
TRIAL = pd.read_sql_query(query, conn)
TRIAL

,Date,Price_Stock_Ex,stock_return_mo,stock_return_pa
0,2024-08,9833.22,-0.075702,-0.611183
1,2024-07,10638.58,-0.000876,-0.010464
2,2024-06,10647.91,0.023790,0.325964
3,2024-05,10400.48,0.035312,0.516552
4,2024-04,10045.74,0.098808,2.097851
...,...,...,...,...
75,2018-05,1006.52,-0.034819,-0.346408
76,2018-04,1042.83,-0.092639,-0.688569
77,2018-03,1149.30,-0.033804,-0.338113
78,2018-02,1189.51,-0.004836,-0.056509


In [13]:
df_bitcoin

,Date,Price_Bitcoin,btc_return_mo,btc_return_pa
0,2024-09,63785.09,0.081934,1.572833
1,2024-08,58954.67,-0.086823,-0.663752
2,2024-07,64559.96,0.042888,0.655212
3,2024-06,61904.96,-0.084658,-0.65406
4,2024-05,67630.41,0.129642,3.318071
...,...,...,...,...
2325,2018-01,16200,-0.033023,-0.331665
2326,2018-01,16753.24,0.120892,2.933362
2327,2018-01,14946.35,-0.007232,-0.083414
2328,2018-01,15055.23,0.018305,0.243179


In [14]:
df_cpi

,Date,Inflation
0,2024-08,0.005156
1,2024-07,0.006023
2,2024-06,0.007141
3,2024-05,0.007498
4,2024-04,0.007581
...,...,...
75,2018-05,0.001264
76,2018-04,0.001224
77,2018-03,0.001144
78,2018-02,0.001194
